# Members: Jace Nina,Kervin Hyka,Mohammed Uddin 


In [ ]:
import sqlite3
from sqlite3 import Error
import random

#### Creating a Database

In [ ]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

#### Creates Student, Enrolled and Courses tables

In [ ]:
def create_table(conn):
    if conn is not None:
        student_table = """CREATE TABLE IF NOT EXISTS  Students (
                            sid integer PRIMARY KEY,
                            sname text NOT NULL,
                            age integer NOT NULL
                            );"""
        
        courses_table = """CREATE TABLE IF NOT EXISTS Courses(
                            cid integer PRIMARY KEY,
                            cname text,
                            credits integer
                            );"""

        enrolled_table = """CREATE TABLE IF NOT EXISTS Enrolled(
                            sid integer,
                            cid integer,
                            grade integer,
                            FOREIGN KEY (sid) REFERENCES Students (sid)
                            FOREIGN KEY (cid) REFERENCES Courses (cid)
                            );"""
        try:
            c = conn.cursor()
            c.execute(student_table)
            c.execute(courses_table)
            c.execute(enrolled_table)
        except Error as e:
            print(e)
    else:
        print("Con not create the database connection")


#### Crates new student record

In [ ]:
def create_student(conn, sid, sname, age):
    sql = '''INSERT INTO Students(sid, sname, age)
            VALUES(?,?,?)'''
    values = sid,sname,age
    cur = conn.cursor()
    cur.execute(sql,values)
    conn.commit()
    cur.close()


### Creates new course record

In [ ]:
def create_course(conn,cid,cname,credit):
    sql = '''INSERT INTO Courses(cid,cname,credits)
             VALUES(?,?,?)
             '''
    values = cid,cname,credit
    cur = conn.cursor()
    cur.execute(sql,values)
    conn.commit()
    cur.close()
    

### Creates new enrollment

In [ ]:
def create_enrollment(conn,sid,cid,grade):
    sql = '''INSERT INTO Enrolled(sid,cid,grade)
             VALUES(?,?,?)'''
    values = sid,cid,grade
    cur = conn.cursor()
    cur.execute(sql,values)
    conn.commit()
    cur.close()

### Delete Enrollment

In [ ]:
def delete_enrollment(conn,sid,cid):
    sql = 'DELETE FROM Enrolled WHERE sid = ? AND cid = ?'
    values = sid,cid
    cur = conn.cursor()
    cur.execute(sql,values)
    conn.commit()
    cur.close()

### Retrieve  a student from the database

In [ ]:
def get_student(conn,sid):
    sql = "SELECT sid,sname,age FROM Students WHERE sid = ?"
    cur = conn.cursor()
    cur.execute(sql,sid)
    stud = cur.fetchall()[0]
    cur.close()
    return str(stud).replace("(","").replace("'","").replace(",","").replace(")","")


### Checks if a students exists in the database

In [ ]:
def sid_exists(conn, sid):
    sql = "SELECT count(*) FROM Students WHERE sid = ?"
    cur = conn.cursor()
    cur.execute(sql,sid)
    if (cur.fetchall()[0] > (0,)):
        cur.close()
        return True
    else:
        cur.close()
        return False

    
        

#### Checks if a Course exists in the database

In [ ]:
def cid_exists(conn, cid):
    sql = "SELECT count(*) FROM Courses WHERE cid = ?"
    cur = conn.cursor()
    cur.execute(sql,(cid,))
    if (cur.fetchall()[0] > (0,)):
        cur.close()
        return True
    else:
        cur.close()
        return False

#### Check if Enrolled table is empty

In [ ]:
def check_enrolled_empty(conn):
    sql = "SELECT count(*) FROM Enrolled"
    cur = conn.cursor()
    cur.execute(sql)
    if(cur.fetchall() == [(0,)]):
        return True
    else:
        return False

#### Check if Courses table is empty

In [ ]:
def check_courses_empty(conn):
    sql = "SELECT count(*) FROM Courses"
    cur = conn.cursor()
    cur.execute(sql)
    if(cur.fetchall() == [(0,)]):
        return True
    else:
        return False

#### Check if the student is already enrolled in the course

In [ ]:
def check_already_enrolled(conn,sid,cid):
    sql = "SELECT count(*) FROM Enrolled WHERE sid = ? and cid = ?"
    values = (sid, cid)
    cur = conn.cursor()
    cur.execute(sql,values)
    if(cur.fetchall()[0][0] > 0):
        return True
    else:
        return False


#### List All Record in Course Table

In [ ]:
def list_all_courses(conn):
    sql = "SELECT * FROM Courses"
    cur = conn.cursor()
    cur.execute(sql)

    courses = cur.fetchall()

    if(courses == []):
        print("Error: No courses are available")
    else:
        print("List of all available courses")
        print("Course ID\t|\tCourse Name\t|\tCourse Credits")
        for courses in courses:
            print(str(courses).replace("(","").replace("'","").replace(",","\t|\t").replace(")",""))

#### Check if student is enrolled in all courses

In [ ]:
def enrolled_all_courses(conn,sid):
    sqlEn = "SELECT count(*) FROM Enrolled WHERE sid = ?"
    sqlCourse = "SELECT count(*) FROM Courses"
    curEn = conn.cursor()
    curCourse = conn.cursor()
    curCourse.execute(sqlCourse)
    curEn.execute(sqlEn,(sid,))

    enrolled = curEn.fetchall()
    course = curCourse.fetchall()

    if(enrolled == course and course != [(0,)]):
        return True
    else:
        return False


#### Enrollment for a student

In [ ]:
def enroll_in_course(conn, sid):
    if check_courses_empty(conn):
        print("Error: There are no available courses to enroll in")
    elif(enrolled_all_courses(conn,sid)):
        print("Error: You are already enrolled in all courses")
    else:
        while True:
            try:
                cid = int(input("Enter a course id: "))
                if cid_exists(conn, str(cid)):
                    if check_already_enrolled(conn, sid, str(cid)):
                        print("Error: You are already enrolled in this course")
                    else:
                        create_enrollment(conn, sid, str(cid), "")
                        print("Enrollment created successfully")
                        break
                else:
                    print("Error: This course does not exist")
            except ValueError:
                print("Error: Enter a valid course id (integer)")
                break


#### Search Courses

In [ ]:
def search_course(conn):
    cname = input("Enter course name")

    sql = 'SELECT * FROM Courses WHERE cname LIKE \"%?%\"'
    cur = conn.cursor()
    cur.execute(sql.replace("?",cname))

    course = cur.fetchall()
    if(course == []):
        print("Error: No such course exist")
    else:
        print("Course ID\t|\tCourse Name\t|\tCourse Credits")
        for course in course:
            print(str(course).replace("(","").replace("'","").replace(",","\t|\t").replace(")",""))



#### List of courses taken by an active student

In [ ]:
def list_my_courses(conn,sid):
    sql = "SELECT c.cid, c.cname, c.credits FROM Enrolled as e, Courses as c WHERE e.cid = c.cid AND e.sid = ?"
    cur = conn.cursor()
    cur.execute(sql,str(sid))
    course = cur.fetchall()

    if(course == []):
        print("Error: No such course exist")
    else:
        print("Course ID\t|\tCourse Name\t|\tCourse Credits")
        for course in course:
            print(str(course).replace("(","").replace("'","").replace(",","\t|\t").replace(")",""))


#### Check if Student is enrolled in a course

In [ ]:
def check_student_enrolled_in_course(conn,sid):
    sql = "SELECT count(*) FROM Enrolled WHERE sid = ?"
    cur = conn.cursor()
    cur.execute(sql,(sid,))
    if(cur.fetchall()[0][0] > 0):
        return True
    else:
        return False

#### Withdraw from a course

In [ ]:
def withdraw(conn, sid):
    if check_courses_empty(conn):
        print("There are no courses to withdraw from")
    elif check_student_enrolled_in_course(conn, sid):
        print("Withdraw from a course")

        while True:
            cid = input("Enter a course id")
            if cid_exists(conn, cid):
                if check_already_enrolled(conn, sid, cid):
                    break
                else:
                    print("You are not enrolled in this course")
            else:
                print("Error: Course does not exist")

        delete_enrollment(conn, sid, cid)
        print("Withdrawn from course successfully")
    else:
        print("You are not enrolled in any courses")


### Adding 5 students to database

In [ ]:
def add5_students(conn):
    names = ['Makena Rosário','Jesse Kjellsson','Anđelko Jedynak','Pomponius Payton',
             'Walther Alexandersson', 'Aviya Albuquerque', 'Priyanka Béranger', 'Elia Baldinotti']
    courseName = ['Psycholinguistics','Algebraic Computation','Sonochemistry','Historical Geography','Combinatorial Mathematics',
    'Narratology and Storytelling','Managerial Economics','Leadership Development','Conspiracy Theories', 'Behavioral Neuroscience']
    unique_ids = random.sample(range(1, 11), 5)
    cidarr = []
    for x in unique_ids:
        sid = random.randint(1,10)
        age = random.randint(20,30)
        num = random.randint(0,5)
        sname = names[num]
        create_student(conn, sid, sname,age)

        cid = random.randint(1024,5024)
        cidarr.append(cid)
        cnum = random.randint(0, len(courseName) - 1)
        cname = courseName[cnum]
        credits = random.randint(2,4)
        create_course(conn,cid,cname,credits)

    for i in unique_ids:
        for j in cidarr:
            grade = random.randint(60,100)
            create_enrollment(conn,i,j,grade)




#### Get student name only

In [ ]:
def get_student_name(conn,sid):
    sql = "SELECT sname FROM Students WHERE sid = ?"
    cur = conn.cursor()
    cur.execute(sql,(sid,))
    stud = cur.fetchall()[0]
    cur.close()
    return str(stud).replace("(","").replace("'","").replace(",","").replace(")","")

#### Create a new student entry

In [ ]:
import re


def add_new_student(conn):
    #Get Student ID number
    while(True):
        try:
            sid = int(input("Enter an student id number"))

            if(sid_exists(conn,str(sid))):
                print("Error: Student id already exist for a different student")
            elif(sid < 0):
                print("Error: Student id must be a positive number")
            else:
                break
        except:
            print("Error: Enter a valid student ID number1")
    #Get Student Name
    while(True):
        try:
            sname = input("Enter your name")
            if(bool(re.match('[a-zA-Z\s]+$', sname))):
                break
            else:
                print("Error: Name must be letters from A-Z")
        except:
            print("Error: Enter a valid name")

    while(True):
        try:
            age = int(input("Enter your age"))
            if(age < 0):
                print("Error: Enter a positive age")
            elif(age > 17 and age < 70):
                break
            else:
                print("Please enter a valid age")
        except:
            print("Error: Enter a valid age between 17 and 70")

    create_student(conn,sid,sname,age)
    print("Student account successfully created")


#### User Interface Start UP

In [ ]:
def main():

    database = r"student_management_system.db"

    #create the database
    conn = create_connection(database)

    #create the tables
    if conn is not None:
        create_table(conn)
    else:
        print("Error cannot connect the database connection.")

    # Use this function only when there is no database created already or only when the database is completely empty
    #add5_students(conn)


    print("Welcome to Montclair State University Registration System")
    while(True):
        try:
            sid = int(input("For existing students\n"+
                        "Enter your student Id\n"+
                        "For new students\n"+
                        "Enter -1 to register\n"
                        ))
            if (sid == -1):
                add_new_student(conn)
            elif(sid_exists(conn,str(sid))):
                break
            else:
                print("Student id does not exist in the database")
        except:
            print("Error: Enter a valid student id number")

    print("Welcome Back to Montclair State University Registration System! ",get_student_name(conn,sid))
    while True:

        user_input = input("Enter a command\n"+
                "L – List: lists all records in the course table\n" +
                "E – Enroll: Enroll in a course\n"+
                "W – Withdraw: Withdraw yourself from a course \n"+
                "S – Search: Search for a course by name\n"+
                "M – My Classes: lists all my classes\n"+
                "X – Exit: exit application\n"
                            )
        if user_input == "L":
            list_all_courses(conn)
        elif user_input == "E":
            enroll_in_course(conn, sid)
        elif user_input == "W":
            withdraw(conn, sid)
        elif user_input == "S":
            search_course(conn)
        elif user_input == "M":
            list_my_courses(conn, sid)
        elif user_input == "X":
            print("Thank You for using Montclair State University Registration System")
            break
        elif user_input == "":
            print("Error: Enter a valid command")

    

In [ ]:
if __name__ == '__main__':
    main()